## Final Project: Modelling the blood pressure curve and RSA (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5487539/)


### Summary

**1. Goal**: "To produce a mathematical model capable of accurately reproducing the dominant features of the blood pressure curve and in particular the dicrotic notch (a small dip in the blood pressure curve associated with aortic valve closure), and the variation due to respiratory sinus arrhythmia (RSA). [**WHY**?] Abnormalities with the dicrotic notch may indicate problems with the valve or poor vascular resistance caused, for example, by severe septic shock"

**2. Compartment models** 
- Initially dividing cardiovascular system into the heart, veins and arteries, where the change in volume of each zone is determined by differential equations measuring the influx and efflux of blood. 

- However, they wanted to show the **dicrotic notch**: *"pressure fluctuation seen when the aortic valve closes. This closure is caused by the **pressure drop** across the valve becoming negative"*.

- They made a new 4-compartment model: Exit region close to the valve or aortich arch (e), left valve (LV), arteries (a), veins (v). $Q_{x}$ is the flow going out of x, and $\dot V_{x}$ represents the change of volume over time, or the derivative with respect to time. 

    --> The rate of **change of volume in the exit region** depends on the difference between the rates at which fluid flows in from the left ventricle and fluid flows out of the aortic arch.
    $$\dot V_e = Q_{LV} - Q_{e}$$
    --> **In the arteries** the volume increases due to fluid coming from the arch and decreases as the fluid goes to the veins
    $$\dot V_a = Q_{e} - Q_{a}$$
    --> **For the veins**, the volume increases due to fluid from the arteries and then as the fluid goes to the ventricle
    $$\dot V_v = Q_{a} - Q_{v}$$
    --> Finally, in the case of the **left ventricle**, the blood comes from the veins and leaves to the aortic arch to start the cycle again
    $$\dot V_{LV} = Q_{v} - Q_{LV}$$

**3. Principles**
- "Changes in pressure are associated with the compliance of blood vessels rather than the relatively small compressibility of blood itself"
- Some definitions:
    - Compliance: The change in volume produced per unit of pressure exerted 
    - Elastance: The change of pressure required to change a unit of volume
- The volume of a compliant elastic vessel can be measured as:
    $$V = V_0 + Cp$$
   where $V_0$ is the volume at pressure time 0 and $C$ is the compliance, a constant, with $V > 0$ (we will assume that the vessel cannot collapse for practicality purposes). 
- Given that the amount of blood that is pumped out of the left ventricle depends on its elastance, we will define the volume as it as:
    $$V_{LV} = V_0 + \frac{p_{LV}}{E_{LV}}$$
    where $E_{LV}$ represents the elastance of the left ventricle. This leads us to define the elastance as the change in pressure over the change in volume.

- As we differentiate the compliance and elastance equations, we use the compliance to describe the change in volume for the blood vessels, and the elastance to define the change in volume of the heart ventricle. Therefore, we get the following values for the volume of each component in terms of pressure:
    $$\dot V_{LV} = \frac{d}{dt}(\frac{p_{LV}}{E_{LV}})$$
    $$\dot V_{v} = C_v \dot p_v$$
    $$\dot V_{a} = C_a \dot p_a$$
    $$\dot V_{e} = C_e \dot p_e$$
    
    _Note on these equations_: The compliance is a constant, whereas the elastance is a variable that varies with time.
- We write the blood flow as standard and uni-directional, using the following equation
    $$Q = \frac{\Delta p}{R}$$
    where R is the resistance to flow, representing all the intricacies that make the blood flow turbulent in a real life scenario. 
    

**4. Basic model**
- Using the equations from the Principles section, we get to the following basic set of equations:
$$\dot V_e \rightarrow C_e \dot p_e = \frac{p_{LV} - p_e}{R_e} - \frac{p_e - p_a}{R_a}$$
$$\dot V_a \rightarrow C_a \dot p_a = \frac{p_e - p_a}{R_a} - \frac{p_a - p_v}{R_v}$$
$$\dot V_v \rightarrow C_v \dot p_v = \frac{p_a - p_v}{R_v} - \frac{p_v - p_{LV}}{R_{LV}}$$
$$\dot V_{LV} \rightarrow \frac{d}{dt}(\frac{p_LV}{E_{LV}}) = \frac{p_v - p_{LV}}{R_{LV}} - \frac{p_{LV} - p_e}{R_e}$$

In [43]:
### Initialize all terms/constants from the paper (Table 1)

# Compliance
C_e = 1.5
C_v = 50
C_a = 1.5

# Resistance
R_a = 0.06
R_v = 0.016
R_LV = 1.2

## PROBLEM!
R_eM = 10 # WHAT IS THIS FOR???? 

# Period of heart beat cycle in seconds
T = 0.9
T_dia = 2*T/3
T_sys = T/3

# Diastolic elastance 
E_d = 0.06

# Variables for systolic elastance
w_0 = 7.54
E_s0 = 3.0
c_1 = 0.1
c_2 = 6
c_3 = 0.01

# Variables for pressure over time f(t) and resistance of the aortic valve R_e
e = 0.00001 
R_e0 = 0.016
A_1 = 0.5
c_4 = 500
c_5 = 4*log(100)
c_6 = 7.5


In [20]:
# Initial values from patient 1 gotten from the paper (Table 3)
p_e0, p_a0, p_v0, p_LV0, delta_t0 = 49.896, 55.000, 16.628, 49.714, 0.75786

**5. Refinements**
- What we are missing: 
    - Driving mechanism for the blood flow
    - Mechanism describing dicrotic notch (The dip in pressure after the aortic valve closes)
    - Mechanism describing aortic valve. (How it opens and closes)
- How to fix it:
    1. Define elastance to model blood flow, as it represents the pumping of the heart, with the constant $E_d$ representing the diastolic elastance (when the blood pressure is the lowest, the amount of pressure in between beats) and $E_s$ representing systolic elastance (maximum pressure when the beat occurs). 
    $$E_{LV} = E_d + a(t)(E_s - E_d)$$
    The value of $a(t)$ reflects the case when $a(t)$ gets switched off for $2/3$ of the cycle, and $T$ represents a full cycle (heart beat). Specifically, $T_{sys} \approx \frac{1}{3}T$. Then:
    $$a(t) = \begin{cases}
        sin^2(\frac{3 \omega (t-t_0)}{2}), & t-t_0 \in (0, T_{sys}) \\
        0, & t-t_0 \in (T_{sys}, T)\end{cases}$$



    - Set T to be time dependent, to ease the modelling of RSA, which associates heart rate with respiration:
    $$\omega = \omega_0 + c_3 sin(\frac{\omega_0 t}{c_2})$$
    Then, because the systolic elastance changes over time with the presence of RSA, we model it as:
    $$ E_s = E_{s0} + c_1 sin(\frac{\omega_0 t}{c_2})$$
    where $c_1$ is half the variation of elastance height, $c_2$ is the number of heart beats per respiration, and $c_3$ is half the variation of $\omega$. Finally, $w_0 = \frac{2\pi}{60}\times HR$ is an angular frequency and $HR$ is the average heart rate in beats per minute.  
   
   

In [69]:
# Define t_0
t_0 = 0

In [70]:
# Define E_s and w

def wf(self, t):
    return w_0 + c_3*sin((w_0*t)/c_2) 

w = function("w", nargs=1, eval_func=wf)

def E_sf(self, t):
    return E_s0 + c_1*sin((w_0*t)/c_2)

E_s = function("E_s", nargs=1, eval_func=E_sf)

In [71]:
# # Define a(t)
def af(self, t):
    if t - t_0 < T_sys and t - t_0 > 0:
        return (sin((3*w(t)*(t-t_0))/2))^2
    elif t - t_0 > T_sys and t - t_0 < T:
        return 0

a = function("a", nargs=1, eval_func=af)


# a = piecewise([[(0, T_sys), (sin((3*w(t)*(t-t_0))/2))^2],[(T_sys, T), 0]]) 

In [72]:
# Define E_LV (t)

def E_LVf(self, t):
    return E_d + a(t)*(E_s(t) - E_d)

E_LV = function("E_LVf", nargs=1, eval_func=E_LVf)

 2. To model the aortic valve, we will define the resistances of the aortic valve and the entrance to the ventricle. The former one depends on pressure difference and time, which is why we rewrite it below. The latter one will be just a Heaviside step function (1 if argument is positive, 0 otherwise). It is worth noting that the other resistances are constants (for $v, LV$ and $a$). 
    $$R_e = R_{e0}[1 + \epsilon_1(exp(-A_1(p_{LV} - p_e)))]$$
 where $R_{e0} \ll R_a$ is a constant variable representing when the valve is fully open. The constant $A_1$, when multiplied by the difference in pressures ($p_{LV} - p_e$) should increase fast as the valve closes. Finally, the constant $\epsilon_1 \ll 1$ assures us that the exponential term is small when the difference in pressures is above 0, but is large when the difference in pressures is negative.
    

In [73]:
# Define R_e - the initial values for A_1 and e_1 are taken from the paper
# def R_ef(self, t):
#     return R_e0*(1 + e*(exp(-A_1*(p_LV - p_e))))

# R_e = function("R_e", nargs=1, eval_func=R_ef)

R_e = R_e0*(1 + e*(exp(-A_1*(p_LV - p_e))))

 3. Model dicrotic notch with pressure over time. Using terms to account for the closing of the valve, as well as the delay in closure we are able to see the pressure drop and the dicrotic notch. We will define $f(t)$ to be pressure as a function of time, representing the impulse of blood as:
 $$f(t) = c_4exp(\frac{-c_5(t-t_n -c_6\Delta t)^2}{(\Delta t)^2})$$
 where $c_4$ is the height of the pulse, $c_5$ is the sharpness and $c_6$ the position of the centre in the wave reading. $\Delta t$ will represent the "delay in closure after the pressure drop becomes negative".

In [74]:
# Define f

t = var("t")

# # PROBLEMS HERE!!!!!!!: 
# # # # # # I don't really know how to represent delta_t, is it just t-t0?
# # # # # # I think that t_n is the amount of periods that have happened? so t/T?
# # # # # # And I don't really know where to use \Delta t0??? 

def t_nf(self, t): # How many beats have we had so far 
    return t/T
    
t_n = function("t_n", nargs=1, eval_func=t_nf)

def delta_tf(self, t):
    return t - t_0

delta_t = function("delta_tf", nargs=1, eval_func=delta_tf)

# Return t_n
def ff(self, t):
    return c_4*exp((-c_5*(t - t_n(t) - c_6*(delta_t(t))^2)/((delta_t(t))^2)))

f = function("f", nargs=1, eval_func=ff)

**6. Final model**
- Combining the equations from the basic model, we refine the volume calculation of the aortic valve and the left ventricle:

$$C_e \dot p_e = \frac{p_{LV} - p_e}{R_e(t)} - \frac{p_e - p_a}{R_a} + f(t)$$
$$C_a \dot p_a = \frac{p_e - p_a}{R_a} - \frac{p_a - p_v}{R_v}$$
$$C_v \dot p_v = \frac{p_a - p_v}{R_v} - \frac{p_v - p_{LV}}{R_{LV}}$$
$$\frac{d}{dt}(\frac{p_{LV}}{E_{LV}}) = \frac{p_v - p_{LV}}{R_{LV}} - \frac{p_{LV} - p_e}{R_e(t)} - f(t)$$

where $f(t)$ represents the flux back through the valve before it closes, which then comes out through the left ventricle to be distributed again.

In [75]:
# FINAL MODEL 

# Define right side for v, e, a and LV

p_LV, p_e, p_v, p_a = var('p_LV, p_e, p_v, p_a') # Pressure
t = var('t') # Time

pe_rhs = ((p_LV - p_e)/R_e - (p_e - p_a)/R_a + f(t))/C_e
pa_rhs = ((p_e - p_a)/R_a - (p_a - p_v)/R_v)/C_a
pv_rhs = ((p_a - p_v)/R_v - (p_v - p_LV)/R_LV)/C_v
pLV_rhs = E_LV(t)*((p_v - p_LV)/R_LV - (p_LV - p_e)/R_e - f(t)) # This is d/dt (p_LV/E_LV), therefore dp_LV/dt = E_LV * d/dt (p_LV/E_LV) I presume


In [76]:
rhs = [pe_rhs, pa_rhs, pv_rhs, pLV_rhs]#, E_s, R_e, delta_t]

In [77]:
# Let's see the results over time

tps = srange(0, 120.0, T) # is the rate T or delta_t0?
initconds = [p_e0, p_a0, p_v0, p_LV0]#, E_s0, R_e0, delta_t0]

# Solve differential equations - make a simulation over time
sol = desolve_odeint(
    rhs,
    ics=initconds,
    times=tps,
    dvars=[p_e, p_a, p_v, p_LV])#, E_s, R_e, delta_t])#,
    #ivar=t)

TypeError: unable to simplify to float approximation

In [24]:
sx = sol[:,0]
sy = sol[:,1]
sz = sol[:,2]
so = sol[:,3]

# plot all three time courses together
p1 = line(zip(tps, sy), color='red')
# p2 = line(zip(tps, sy), color='blue')
# p3 = line(zip(tps, sz), color='green')
# p4 = line(zip(tps, so), color='brown')

show(p1)# + p2 + p3 + p4)

NameError: name 'sol' is not defined

**7. Sensitivity Analysis**

- There are 6 parameters that make a big difference in the L2 distance between the simulated pressure and the real data. We write the original variables in decreasing order of importance, and in parenthesis we write the variables that depend on these, and therefore share importance. These are:
    - $E_d (\text{and therefore } C_v)$
    - $E_{s0} (\text{similar with } C_a \text{ and } C_e)$
    - $p_a(0) (\text{and } p_e(0), p_{LV}(0))$
    - $A$
    - $R_{LV}$
    - $\omega_0$